### Cat and Dog Classification

In [1]:
# import libraries
import torch
from torch.optim import Adam
from torch.nn import Linear, Sigmoid, BCELoss, Module, Softmax, ReLU, CrossEntropyLoss, Conv2d, MaxPool2d
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder

from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split

In [2]:
# load data
transform = transforms.Compose([
    transforms.Resize((98, 98)),
    transforms.ToTensor()
])

dataset = ImageFolder(root='data', transform=transform)

print(dataset, '\n')
print('- number of images:', len(dataset))
print('- class:', dataset.classes)

dataloader = DataLoader(dataset, batch_size=20, shuffle=True)

for x_data, y_data in dataloader:
    print('- x_data size:', x_data.shape)
    print('- y_data:', y_data)

Dataset ImageFolder
    Number of datapoints: 20
    Root location: data
    StandardTransform
Transform: Compose(
               Resize(size=(98, 98), interpolation=PIL.Image.BILINEAR)
               ToTensor()
           ) 

- number of images: 20
- class: ['cat', 'dog']
- x_data size: torch.Size([20, 3, 98, 98])
- y_data: tensor([0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0])


In [9]:
example_img = torch.zeros(1, 3, 98, 98)

conv1 = Conv2d(in_channels=3, out_channels=3, kernel_size=5, stride=1)
x = conv1(example_img)

conv2 = Conv2d(in_channels=3, out_channels=10, kernel_size=5, stride=1)
x = conv2(x)

max1 = MaxPool2d(kernel_size=2, stride=1)
x = max1(x)

print(x.shape)
x = torch.flatten(x, 1)
x.shape[1]

torch.Size([1, 10, 89, 89])


79210

In [10]:
# CNN Class
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = Conv2d(in_channels=3, out_channels=3, kernel_size=5, stride=1)
        self.conv2 = Conv2d(in_channels=3, out_channels=10, kernel_size=5, stride=1)
        self.max1 = MaxPool2d(kernel_size=2, stride=1)
        self.fc1 = Linear(10 * 89 * 89, 36)
        self.fc2 = Linear(36, 2)
        self.smax = Softmax(dim=1)
    
    def get_flatten_size(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.max1(x)
        # get 10 * 89 * 89
        x = torch.flatten(x, 1)
        return x.shape[1]
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.max1(x)
        x = x.view(-1, 10 * 89 * 89)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.smax(x)
        return x